# Heston Model: an Integral Approach

In this notebook, we use an alternative approach to the Heston model.

In [22]:
import numpy as np
import pandas as pd
from scipy.integrate import quad

Heston’s Stochastic Volatility Model under real world probability measure:

$$\begin{aligned}
&d S_{t}=\mu S_{t} d t+\sqrt{v_{t}} S_{t} d W_{1, t}^{\mathbb{P}} \\
&d v_{t}=\kappa\left(\theta-v_{t}\right) d t+\sigma \sqrt{v_{t}} d W_{2, t}^{\mathbb{P}} \\
&\rho d t=d W_{2, t}^{\mathbb{P}} d W_{2, t}^{\mathbb{P}}
\end{aligned}$$

Using Girsanov's Thereom to $\mathbb{P} \rightarrow \mathbb{Q}$:

$$
\begin{aligned}
&d W_{S, t}^{\mathbb{Q}}=d W_{S, t}^{\mathbb{P}}+\alpha_{S} d t, \alpha_{S}=\frac{\mu_{\mathrm{p}}-r}{v_{t}} \\
&d W_{v, t}^{\mathbb{Q}}=d W_{v, t}^{\mathbb{P}}+\alpha_{v} d t, \alpha_{v}=\frac{\lambda}{\sigma^{\mathbb{P}}} \sqrt{v_{t}}
\end{aligned}
$$

We then obtain Heston’s Stochastic Volatility Model under risk-neutral measure:

$$
\begin{aligned}
&d S_{t}=r S_{t} d t+\sqrt{v_{t}} S_{t} d W_{1, t}^{\mathbb{Q}} \\
&d v_{t}=\kappa^{\mathbb{Q}}\left(\theta^{\mathbb{Q}}-v_{t}\right) d t+\sigma \sqrt{v_{t}} d W_{2, t}^{\mathbb{Q}} \\
&\rho^{\mathbb{Q}} d t=d W_{2, t}^{\mathbb{Q}} d W_{2, t}^{\mathbb{Q}}
\end{aligned}
$$

$$
\rho^{\mathbb{Q}}=\rho, \kappa^{\mathbb{Q}}=\kappa+\lambda, \theta^{\mathbb{Q}}=\kappa \theta /(\kappa+\lambda)
$$

Notations:

*   $S(t)$: Equity spot price, financial index
*   $V(t)$: Variance
* $C$: European call option price
* $K$: Strike price
* $W_{1,2}$: Standard Brownian movements
*$r$: Interest rate
*$q$: Dividend yield
*$\kappa $: Mean reversion rate
*$θ$: Long run variance
*$V_0$: Initial variance
*$σ$: Volatility of variance
*$ρ$: Correlation parameter
*$t_0$: Current date
*$T$: Maturity date
*$\lambda$: Variance risk premium


The paper [Heston’s Stochastic Volatility
Model Implementation,
Calibration and Some Extensions](https://www.maths.univ-evry.fr/pages_perso/crepey/Finance/051111_mikh%20heston.pdf) provided the following solution to Heston model. 

First, using standard arbitrage arguments we arrive at Garman’s partial differential equation:
$$
\frac{\delta C}{\delta t}+\frac{S^{2} v}{2} \frac{\delta^{2} C}{\delta S^{2}}+r S \frac{\delta C}{\delta S}-r C+[\kappa(\theta-v)-\lambda v] \frac{\delta C}{\delta v}+\frac{\sigma^{2} v}{2} \frac{\delta^{2} C}{\delta v^{2}}+\rho \sigma S v \frac{\delta^{2} C}{\delta S \delta v}=0
$$ 

Using method of characteristic functions, we bulit a solution to the above PDE in a similar form to the  Black-Scholes model:

$$
C\left(S_{0}, K, v_{0}, \tau\right)=S P_{1}-K e^{-r \tau} P_{2}
$$
where
- $P_{1}$ is the delta of the European call option and
- $P_{2}$ is the conditional risk neutral probability that the asset price will be greater than $\mathrm{K}$ at the maturit
$y .$

Given characteristic functions $\varphi_{1,2}$, both probabilities $P_{1,2}$ satisfy Garman’s PDE.  $P_{1,2}$ defined via the inverse Fourier transformation:

$$
\begin{aligned}
&X=\ln (S) \\
&P_{j}=\frac{1}{2}+\frac{1}{\pi} \int_{0}^{\inf } \Re\left[\frac{e^{-i \phi \ln K} \varphi_{j}\left(X_{0}, K, v_{0}, \tau ; \phi\right)}{i \phi}\right] d \phi, j \in\{1,2\}
\end{aligned}
$$

While the characteristic functions have the form:

$\varphi_{j}\left(X_{0}, K, v_{0}, \tau ; \phi\right)=e^{C(\tau ; \phi)+D(\tau ; \phi) v+i \phi X},\ j = 1,2$


With:

$\begin{aligned}
&\left.C(\tau ; \phi)=r \phi i \tau+\frac{a}{\sigma^{2}}\left[\left(b_{j}-\rho \sigma \phi i+d\right) \tau-2 \ln \left[\frac{1-g e^{d \tau}}{1-g}\right]\right)\right] \\
&D(\tau ; \phi)=\frac{b_{j}-\rho \sigma \phi i+d}{\sigma^{2}}\left[\frac{1-e^{d \tau}}{1-g e^{d \tau}}\right] \\
&-g=\frac{b_{j}-\rho \sigma \phi i+d}{b_{j}-\rho \sigma \phi i-d} \\
&-d=\sqrt{\left(\rho \sigma \phi i-b_{j}\right)^{2}-\sigma^{2}\left(2 u_{j} \phi i-\phi^{2}\right)} \\
&-u_{1}=0.5, u_{2}=-0.5-a=\kappa \theta \\
&-b_{1}=\kappa+\lambda-\rho \sigma \\
&-b_{2}=\kappa+\lambda
\end{aligned}$

## Implement the characteristic function

$\varphi\left(X_{0}, K, v_{0}, \tau ; \phi\right)=e^{r \phi i \tau} S^{i \phi}\left[\frac{1-g e^{d \tau}}{1-g}\right]^{\frac{2 a}{\sigma^{2}}} \exp \left[\frac{a \tau}{\sigma^{2}}\left(b_{2}-\rho \sigma \phi i+d\right)+\frac{v_{0}}{\sigma^{2}}\left(b_{2}-\rho \sigma \phi i+d\right)\left[\frac{1-e^{d \tau}}{1-g e^{d \tau}}\right]\right]$

where d and g no longer change with b1, b2 or u1, u2.


$\begin{aligned}
-\ d &=\sqrt{(\rho \sigma \phi i-b)^{2}+\sigma^{2}\left(\phi i+\phi^{2}\right)} \\
-\ g &=\frac{b-\rho \sigma \phi i+d}{b-\rho \sigma \phi i-d} \\
-\ a &=\kappa \theta \\
-\ b &=\kappa+\lambda
\end{aligned}$

In [2]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  """
  Returns the result of implementing the characteristic function.

  Args:
  - phi: variable in characteristic function
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  a = kappa*theta
  b = kappa+lambd
  print(np.shape(rho), np.shape(sigma), phi)
  rspi = rho*sigma*phi*1j
  d = np.sqrt( (rspi - b)**2 + sigma**2*(phi*1j + phi**2) )
  g = (b - rspi + d)/(b - rspi - d)

  exp1 = np.exp(r*phi*1j*tau)
  term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
  exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
  return exp1*term2*exp2

## Define the integrand as a function
$$
\int_{0}^{\inf } \Re\left[e^{r \tau} \frac{\varphi(\phi-i)}{i \phi K^{i \phi}}-K \frac{\varphi(\phi)}{i \phi K^{i \phi}}\right] d \phi
$$

In [3]:
def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  """
  Returns the result of implementing the integrand.

  Args:
  - phi: variable in characteristic function
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  numerator = np.exp(r*tau) * heston_charfunc(phi-1j, *args) - K * heston_charfunc(phi, *args)
  denominator = 1j*phi*K**(1j*phi)
  return numerator/denominator


## Perform numerical integration over integrand and calculate option price.
$$
C\left(S_{0}, K, v_{0}, \tau\right)=\frac{1}{2}\left(S_{0}-K e^{-r \tau}\right)+\frac{1}{\pi} \int_{0}^{\inf } \Re\left[e^{r \tau} \frac{\varphi(\phi-i)}{i \phi K^{i \phi}}-K \frac{\varphi(\phi)}{i \phi K^{i \phi}}\right] d \phi
$$

Using rectangular integration:

In [4]:
def heston_price(K, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  """
  Return the option price calculated by heston model.

  Args:
  - phi: variable in characteristic function
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  real_integral, err = np.real(quad(integrand, 0, 100, args = args))
  return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

Using `quad` from `scipy.integrate` to integrate:

In [5]:
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    
    P, umax, N = 0, 100, 10000
    dphi=umax/N #dphi is width
    for i in range(1,N):
        # rectangular integration
        phi = dphi * (2*i + 1)/2 # midpoint to calculate height
        numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)
        
        P += dphi * numerator/denominator
        
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

##Test the model with dummy parameters.

In [15]:
S0 = 100. # initial asset price
K = 100. # strike
v0 = 0.1 # initial variance
r = 0.03 # risk free rate
kappa = 1.5768 # rate of mean reversion of variance process
theta = 0.0398 # long-term mean variance
sigma = 0.3 # volatility of volatility
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock process
tau = 1. # time to maturity

heston_price_rec(K, S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

() () (50-1j)
() () 50.0
() () (1.304673574141411-1j)
() () 1.304673574141411
() () (98.69532642585858-1j)
() () 98.69532642585858
() () (6.746831665550772-1j)
() () 6.746831665550772
() () (93.25316833444923-1j)
() () 93.25316833444923
() () (16.029521585048776-1j)
() () 16.029521585048776
() () (83.97047841495123-1j)
() () 83.97047841495123
() () (28.33023029353764-1j)
() () 28.33023029353764
() () (71.66976970646236-1j)
() () 71.66976970646236
() () (42.55628305091844-1j)
() () 42.55628305091844
() () (57.44371694908156-1j)
() () 57.44371694908156
() () (0.21714184870959485-1j)
() () 0.21714184870959485
() () (99.78285815129041-1j)
() () 99.78285815129041
() () (3.4921254322145856-1j)
() () 3.4921254322145856
() () (96.50787456778542-1j)
() () 96.50787456778542
() () (10.959113670679152-1j)
() () 10.959113670679152
() () (89.04088632932084-1j)
() () 89.04088632932084
() () (21.862143266569767-1j)
() () 21.862143266569767
() () (78.13785673343023-1j)
() () 78.13785673343023
() () (35

/usr/local/lib/python3.7/dist-packages/scipy/integrate/quadpack.py:463: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


11.540361819355377

## Implementing Integral Approach in Vector Case

Codes above implemented the case with a single option. In this section, we are going to use heston model for pricing of mulitiple options.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/MyDrive/processed_data.csv')
df.head()

,Unnamed: 0,optionid,securityid,strike,date_traded,contract_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,mean_volatility,reversion,var_of_vol,rho,lambda
0,0,30246878.0,504880.0,27.0,2003-04-24,0.365,23.108701,5109.0,0.405479,0.855878,0.012725,0.344999,0.027174,2.514547,0.136556,-0.332888,0.860253
1,1,30246882.0,504880.0,28.0,2003-04-24,0.225,23.108701,157.0,0.405479,0.825311,0.012725,0.344999,0.035583,1.412536,0.043306,-0.189034,0.831190
2,2,30246886.0,504880.0,29.0,2003-04-24,0.131,23.108701,122.0,0.405479,0.796852,0.012725,0.344999,0.016089,2.366108,0.081914,-0.164918,0.519741
3,3,30246890.0,504880.0,30.0,2003-04-24,0.074,23.108701,6020.0,0.405479,0.770290,0.012725,0.344999,0.004220,0.108479,0.546855,-0.167983,0.899078
4,4,30262356.0,504889.0,2.8,2003-04-24,0.056,2.455500,1000.0,0.405479,0.876964,0.012725,0.265610,0.010153,3.142642,0.640966,-0.430974,1.020605


In [19]:
def heston_price_vec(df):
  """
  Return the option price calculated by heston model.

  Args:
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  S0    = df['underlyings_price'].values 
  v0    = df['volatility'].values 
  r     = df['rate'].values 
  theta = df['mean_volatility'].values 
  kappa = df['reversion'].values 
  sigma = df['var_of_vol'].values 
  lambd = df['lambda'].values
  K     = df['strike'].values 
  rho   = df['rho'].values 
  tau   = df['days_to_maturity'].values

  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  return heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

## Test Implementation in Vector Case

In [17]:
N = 3

S_0 = np.repeat(100.0, N)     # Initial spot price
K = np.repeat(100.0, N)       # Strike price
r = np.repeat(0.03, N)      # Risk-free rate
v_0 = np.repeat(0.1,N)   # Initial volatility 
tau = np.repeat(1.0, N)       # One year until expiry

rho = np.repeat(-0.5711, N)      # Correlation of asset and volatility
kappa = np.repeat(1.5768, N)    # Mean-reversion rate
theta = np.repeat(0.0398,N)    # Long run average volatility
sigma = np.repeat(0.3,N)     # "Vol of vol"  
lambd = np.repeat(0.575,N)    # risk premium of variance


dummy = pd.DataFrame(columns=['underlyings_price', 'strike', 'rate', 
                    'days_to_maturity', 'volatility', 'mean_volatility', 
                    'reversion', 'var_of_vol', 'rho', 'lambda'])
dummy['days_to_maturity'] = tau
dummy['underlyings_price'] = S_0
dummy['volatility'] = v_0
dummy['rate'] = r
dummy['mean_volatility'] = theta
dummy['reversion'] = kappa
dummy['var_of_vol'] = sigma
dummy['strike'] = K
dummy['rho'] = rho
dummy['lambda'] = lambd

dummy.head()

,underlyings_price,strike,rate,days_to_maturity,volatility,mean_volatility,reversion,var_of_vol,rho,lambda
0,100.0,100.0,0.03,1.0,0.1,0.0398,1.5768,0.3,-0.5711,0.575
1,100.0,100.0,0.03,1.0,0.1,0.0398,1.5768,0.3,-0.5711,0.575
2,100.0,100.0,0.03,1.0,0.1,0.0398,1.5768,0.3,-0.5711,0.575


In [21]:
test_output = heston_price_vec(dummy)
test_output

流式输出内容被截断，只能显示最后 5000 行内容。
(3,) (3,) (75.005-1j)
(3,) (3,) 75.005
(3,) (3,) (75.015-1j)
(3,) (3,) 75.015
(3,) (3,) (75.025-1j)
(3,) (3,) 75.025
(3,) (3,) (75.035-1j)
(3,) (3,) 75.035
(3,) (3,) (75.045-1j)
(3,) (3,) 75.045
(3,) (3,) (75.055-1j)
(3,) (3,) 75.055
(3,) (3,) (75.065-1j)
(3,) (3,) 75.065
(3,) (3,) (75.075-1j)
(3,) (3,) 75.075
(3,) (3,) (75.08500000000001-1j)
(3,) (3,) 75.08500000000001
(3,) (3,) (75.095-1j)
(3,) (3,) 75.095
(3,) (3,) (75.105-1j)
(3,) (3,) 75.105
(3,) (3,) (75.115-1j)
(3,) (3,) 75.115
(3,) (3,) (75.125-1j)
(3,) (3,) 75.125
(3,) (3,) (75.135-1j)
(3,) (3,) 75.135
(3,) (3,) (75.145-1j)
(3,) (3,) 75.145
(3,) (3,) (75.155-1j)
(3,) (3,) 75.155
(3,) (3,) (75.165-1j)
(3,) (3,) 75.165
(3,) (3,) (75.175-1j)
(3,) (3,) 75.175
(3,) (3,) (75.185-1j)
(3,) (3,) 75.185
(3,) (3,) (75.19500000000001-1j)
(3,) (3,) 75.19500000000001
(3,) (3,) (75.205-1j)
(3,) (3,) 75.205
(3,) (3,) (75.215-1j)
(3,) (3,) 75.215
(3,) (3,) (75.22500000000001-1j)
(3,) (3,) 75.22500000000001
(3,) (3,) 

array([11.52114509, 11.52114509, 11.52114509])

The codes in this notebook is adapted from [Heston Model Calibration to option prices](https://quantpy.com.au/stochastic-volatility-models/heston-model-calibration-to-option-prices/).